<a href="https://colab.research.google.com/github/Ashish-sinh/recommendation-using-vector-db/blob/main/Save_Vector_Embeddings_in_Qdrant_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers datasets qdrant-client --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
# before run this upload your own kaggle.json for API and download dataset direct from hub
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/

In [3]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d shivamb/fashion-clothing-products-catalog

  0% 0.00/615k [00:00<?, ?B/s]
100% 615k/615k [00:00<00:00, 134MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/fashion-clothing-products-catalog.zip')
zip_ref.extractall()
zip_ref.close()

In [5]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/myntra_products_catalog.csv')
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


## Data Preprocessing :

In [6]:
df.dropna(inplace = True)
df.reset_index(inplace = True , drop= True)

In [7]:
df[['ProductName','ProductBrand','Gender','Description','PrimaryColor']] = df[['ProductName','ProductBrand','Gender','Description','PrimaryColor']].apply(lambda x : x.str.lower())
df.head(2)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,dkny unisex black & grey printed medium trolle...,dkny,unisex,11745,7,"black and grey printed medium trolley bag, sec...",black
1,10016283,ethnovogue women beige & grey made to measure ...,ethnovogue,women,5810,7,beige & grey made to measure kurta with churid...,beige


In [8]:
df = df.drop(columns = ['Price (INR)','NumImages'])
df.head(2)

,ProductID,ProductName,ProductBrand,Gender,Description,PrimaryColor
0,10017413,dkny unisex black & grey printed medium trolle...,dkny,unisex,"black and grey printed medium trolley bag, sec...",black
1,10016283,ethnovogue women beige & grey made to measure ...,ethnovogue,women,beige & grey made to measure kurta with churid...,beige


In [9]:
# remove stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
df[['ProductName','ProductBrand','Gender','Description','PrimaryColor']] = df[['ProductName','ProductBrand','Gender','Description','PrimaryColor']].apply(lambda x : [' '.join(word) for word in x.str.split() if word not in stop_words])
df.head(4)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,ProductID,ProductName,ProductBrand,Gender,Description,PrimaryColor
0,10017413,dkny unisex black & grey printed medium trolle...,dkny,unisex,"black and grey printed medium trolley bag, sec...",black
1,10016283,ethnovogue women beige & grey made to measure ...,ethnovogue,women,beige & grey made to measure kurta with churid...,beige
2,10009781,spykar women pink alexa super skinny fit high-...,spykar,women,pink coloured wash 5-pocket high-rise cropped ...,pink
3,10015921,raymond men blue self-design single-breasted b...,raymond,men,blue self-design bandhgala suitblue self-desig...,blue


In [10]:
df['Context'] = df.apply(lambda row : f'ProductName : {row.ProductName} ,ProductBrand  : {row.ProductBrand} ,Gender : {row.Gender} , Description : {row.Description}  , PrimaryColor : {row.PrimaryColor}' ,axis = 1)
df.head()

,ProductID,ProductName,ProductBrand,Gender,Description,PrimaryColor,Context
0,10017413,dkny unisex black & grey printed medium trolle...,dkny,unisex,"black and grey printed medium trolley bag, sec...",black,ProductName : dkny unisex black & grey printed...
1,10016283,ethnovogue women beige & grey made to measure ...,ethnovogue,women,beige & grey made to measure kurta with churid...,beige,ProductName : ethnovogue women beige & grey ma...
2,10009781,spykar women pink alexa super skinny fit high-...,spykar,women,pink coloured wash 5-pocket high-rise cropped ...,pink,ProductName : spykar women pink alexa super sk...
3,10015921,raymond men blue self-design single-breasted b...,raymond,men,blue self-design bandhgala suitblue self-desig...,blue,ProductName : raymond men blue self-design sin...
4,10017833,parx men brown & off-white slim fit printed ca...,parx,men,"brown and off-white printed casual shirt, has ...",white,ProductName : parx men brown & off-white slim ...


In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer (
    'sentence-transformers/all-MiniLM-L6-v2',
          device='cuda' ) # 'cpu' if you are using low ram or not connected with GPU

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from datasets import Dataset , DatasetDict

# convert pandas dataframe into huggingface Dataset object
hf_dataset = DatasetDict()
hf_dataset['train']= Dataset.from_pandas(df)
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['ProductID', 'ProductName', 'ProductBrand', 'Gender', 'Description', 'PrimaryColor', 'Context'],
        num_rows: 11597
    })
})

In [13]:
from tqdm import tqdm

def generate_embeddings(split , batch_size = 32) :
  embeddings = []
  split_name = 'train'
  split_len = len(split)

  with tqdm(total =split_len  , desc =f'Generating embeddings for {split_name}') as pbar :
    for i in range(0 , split_len , batch_size) :
      batch_sentence = split['Context'][i:i+batch_size]
      batch_embedding = model.encode(batch_sentence)
      embeddings.extend(batch_embedding)
      pbar.update(len(batch_sentence))

  return embeddings


In [14]:
context_embeddings = generate_embeddings(hf_dataset['train'] , batch_size = 32)
hf_dataset['train'] = hf_dataset['train'].add_column('embeddings',context_embeddings)

Generating embeddings for train: 100%|██████████| 11597/11597 [00:31<00:00, 371.24it/s]


In [15]:
from qdrant_client import models, QdrantClient

In [16]:
from qdrant_client import QdrantClient
from google.colab import userdata

# Create Empty space at qdrant cloud database and copy URL :

# {  QDRANT_API_KEY = userdata.get('QDRANT_API_KEY')

# qdrant_client = QdrantClient(
#     url="https://3e9ea0c2-1b73-44e6-a03c-31bd5c5cd570.us-east4-0.gcp.cloud.qdrant.io:6333",
#     api_key= QDRANT_API_KEY ) }

qdrant_client = QdrantClient(":memory:")


In [17]:
# create qdrant Collection for embeddings :

qdrant_client.create_collection(
    collection_name = 'fashion-recommendation' ,
    vectors_config = models.VectorParams (
        size = model.get_sentence_embedding_dimension() ,
        distance = models.Distance.COSINE ,
    ),
)

True

In [18]:
from itertools import islice

def batched(iterable , batch_size ) :
  iterator = iter(iterable)
  while batch := list(islice(iterator , batch_size)) :
    yield batch

batch_size = 100

In [19]:
for batch in batched(hf_dataset['train'] ,batch_size) :
  ids = [sample.pop('ProductID') for sample in batch]
  vectors = [sample.pop('embeddings') for sample in batch ]

  qdrant_client.upsert(
      collection_name = 'fashion-recommendation' ,
      points = models.Batch (
          ids = ids ,
          vectors = vectors ,
          payloads = batch ,
      ),
  )

In [20]:
hits = qdrant_client.search(
    collection_name="fashion-recommendation",
    query_vector=model.encode("black pant").tolist(),
    limit=5
)
for hit in hits:
  print(hit.payload['ProductName'], "score:", hit.score)

hummel men black solid straight-fit track pant score: 0.5317343393626774
panit women pink regular fit solid peg trousers score: 0.4950135044409767
panahi men black mojaris score: 0.4885017053564573
alcis men black solid slim fit training track pants score: 0.48204624158499865
fablestreet women black straight fit solid formal trousers score: 0.4775757150971638
